In [ ]:
//Attributes: Compiler Tricks
using System.Diagnostics;

[Conditional("DEBUG")]
void TraceMethod()
{ }

In [ ]:
var data = new[] { 5, 3, 7, 8, 2, 9 };
foreach(var d in data)
    Console.WriteLine(d);

# Value and Reference Types

- Default Values
- Nullable

In [ ]:
string location;    // reference type
DateTime time;      // value type

Console.WriteLine(location == null ? "location is null" : location);
Console.WriteLine(time == null ? "time is null" : time.ToString());

# Generics

- Classes
- Methods
- Delegate
- Collections

In [ ]:
public class Box<T>
{
    public T Field;
}

void Swap<T>(T left, T right)
{
    T temp = left;
    left = right;
    right = temp;
}


# Delegates

- Multicasting
- Callback & Lazy Loading

In [ ]:
delegate void DelegateString(string s);
delegate string GetStringDelegate();

void Process(GetStringDelegate getServer, DelegateString connected, DelegateString disconnected)
{
    var server = getServer();
    //connect to remote server
    connected($"Connected to {server}");

    //do something
    //disconnect
    disconnected($"Disconnected from {server}");

    //cleanup or something more
}


void print(string s) { Console.WriteLine(s); }
string loadConfig() => "google.com";

Process(loadConfig, print, print);

In [ ]:
//Delegates and Generics
//Action, Func, EventHandler.....and Predicate

var print = new Action<string>((string s) =>
{
    Console.WriteLine(s);
});

Action<string> print2 = s => Console.WriteLine(s);

print("Hello Action");
print2("Hello Action");

In [ ]:
int Calculation(Func<int, int, int> operation, int x, int y)
{
    Console.WriteLine($"Calculation(.., {x}, {y})");
    var r = operation(x, y);
    Console.WriteLine($"Returned {r}");
    return r;
}

Calculation((x, y) => x + y, 2, 3);

In [ ]:
class BinaryTree<T> where T:IComparable<T>
{
    public T Value { get; set; }
    public BinaryTree<T> Left;
    public BinaryTree<T> Right;

    public BinaryTree(T value)
    {
        this.Value = value;
        this.Left = this.Right = null;
    }

    public void Insert(T value)
    {
        Func<BinaryTree<T>, T, BinaryTree<T>> insert = (t, v) =>
        {
            if (t == null)
            {
                t = new BinaryTree<T>(v);
                return t;
            }
            else
            {
                t.Insert(v);
                return t;
            }
        };

        if (value.CompareTo(this.Value) < 0)
            this.Left = insert(this.Left, value);
        else
            this.Right = insert(this.Right, value);
    }

    public IEnumerator<T> GetEnumerator()
    {
        if (this.Left != null)
        {
            foreach (T item in this.Left)
                yield return item;
        }

        yield return this.Value;

        if (this.Right != null)
        {
            foreach (T item in this.Right)
                yield return item;
        }
    }
}

In [ ]:
var values = new[] { 5, 4, 7, 2, 8, 1 };

var tree = new BinaryTree<int>(9);
foreach(var v in values)
    tree.Insert(v);

foreach(var v in tree)
    Console.WriteLine(v);

- Contravariance: Using one method for different event / Delegates
- Covariance: Delegate can be more generatlized say object or base type and method can use specialized types

# Events

In [ ]:
delegate void DelegateString(string s);
event DelegateString StringEvent;               // instead of these two lines; we can simply have event Action<string> StringEvent;

void print(string s) { Console.WriteLine(s); }
void log(string s) { Console.WriteLine($"Logged {s}"); }

StringEvent += new DelegateString(print);
StringEvent += new DelegateString(log);

StringEvent("Its a beautiful day");

In [ ]:
class StepCompletedEventArgs : EventArgs
{
    public bool CanContinue = false;
}

event EventHandler<StepCompletedEventArgs> StepCompleted;

void MultiStepLogic()
{
    //Step 1
    //Before proceeding to step 2; we need to check if we can continue
    var args = new StepCompletedEventArgs();
    if (null != StepCompleted) // someone has subscribed
        StepCompleted(new object(), args);
    
    if (args.CanContinue)
        Console.WriteLine("I can continue");
    else
        Console.WriteLine("I cann't continue");
    //Step 2
}

//Lets subscribe
StepCompleted += (s, e) => e.CanContinue = true;
MultiStepLogic();

In [ ]:
void dontLetThemContinue(object sender, StepCompletedEventArgs args) { args.CanContinue = false; }

//lets make another subscription
StepCompleted += dontLetThemContinue;     // this being second; this will have final say
MultiStepLogic(); // shouldnt continue

StepCompleted -= dontLetThemContinue;     // for unsubscribe to work we had to use local method
MultiStepLogic(); // should continue

In [ ]:
//Inheritance and Events

class BaseClass
{
    event EventHandler<ClickEventArgument> OnClick;

    protected void RaiseEvent(ClickEventArgument args)
    {
        if (null != OnClick) OnClick(this, args);
    }
}

class ChildClass : BaseClass // we have inherited the event
{
    void SomeMethod()
    {
        //if we need to raise the event
        base.RaiseEvent(new ClickEventArgument());
    }
}

In [ ]:
using System.Collections.Generic; //We have tons of Generic Collections in there

var list = new List<int>();
var linkedList = new LinkedList<int>();